## Observação:
Os arquvivoos estão sendo salvos em formato Delta, devido a importancia da garantia da eficiência e confiabilidade dos dados. Dessa forma, temos consultas melhoradas e alterações seguras.

In [0]:
# ====================================================
# 1. Importando bibliotecas
# ====================================================
import requests   # Biblioteca para fazer requisições HTTP e consumir a API SpaceX
import datetime   # Para capturar e formatar a data/hora da ingestão
from pyspark.sql import SparkSession   # Para criar a sessão Spark e manipular DataFrames
from pyspark.sql.functions import col, row_number, when, lit   # Funções SQL do Spark para limpeza/transformação
from pyspark.sql.window import Window   # Para aplicar funções de janela, como row_number()

# Criando sessão Spark
spark = SparkSession.builder.appName("SpaceXCollector").getOrCreate()

# ====================================================
# 2. Função genérica para coletar dados da SpaceX
# ====================================================
def get_spacex_data(endpoint):
    """
    Coleta dados de um endpoint da SpaceX API v4.
    Exemplo de endpoint: 'launches', 'rockets', 'ships'.
    """
    base_url = "https://api.spacexdata.com/v4" # URL base da API
    url = f"{base_url}/{endpoint}" #Monta URL para endpoint

    print(f"📡 Coletando {endpoint} de {url}") #Verifica qual endpoint está sendo coletado
    resp = requests.get(url) # Faz requisição HTTP vi GET

    if resp.status_code != 200: # Verifica se a requisição foi bem sucedida
        print(f"❌ Erro {resp.status_code} ao coletar {endpoint}") # Exibe mensagem de erro
        return [] # Retorna lista vazia

    data = resp.json() # Converte resposta em JSON

    # Adiciona metadata de ingestão
    ingestion_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") # Captura data/hora atual
    if isinstance(data, list): # Verifica se a resposta é uma lista
        for item in data: # Itera sobre cada item da lista
            item["ingestion_date"] = ingestion_date # Adiciona data/hora de ingestão
    else: # Caso contrário, trata como um único item
        data["ingestion_date"] = ingestion_date # Adiciona data/hora de ingestão

    return data # Retorna dados

# ====================================================
# 3. Bronze – coleta dados crus
# ====================================================
endpoints = ["launches", "rockets", "ships"] # Lista de endpoints da API

for ep in endpoints: # Itera sobre cada endpoint
    data = get_spacex_data(ep) # Coleta dados do endpoint

def normalize_data(data):# Normaliza dados
    for item in data:# Itera sobre cada item
        for key, value in item.items(): # Itera sobre cada chave
            # Se o valor for dict ou list, transforma em string JSON
            if isinstance(value, (dict, list)): # Verifica se o valor é dict ou list
                item[key] = json.dumps(value) # Transforma em string JSON
    return data # Retorna dados normalizados


    if len(data) > 0: # Verifica se há dados
        # Cria DataFrame Spark diretamente de lista de dicionários
        df_bronze = spark.createDataFrame(data) 

        # Salva como Delta no catálogo bronze
        df_bronze.write.format("delta").mode("overwrite").saveAsTable(f"workspace.bronze.spacex_{ep}")
        print(f"✅ Bronze salvo para {ep}") # Exibe mensagem de sucesso
    else:
        print(f"⚠️ Nenhum dado retornado para {ep}") # Exibe mensagem de aviso

# ====================================================
# 4. Silver – limpeza básica (exemplo para lançamentos)
# ====================================================
df_launches = spark.table("workspace.bronze.spacex_launches") # Carrega dados do catálogo bronze

df_silver_launches = ( 
    df_launches
    .select( #Seleciona apenas colunas relevantes e renomea as colunas
        col("id").alias("launch_id"),
        col("name").alias("mission_name"),
        col("date_utc"),
        col("rocket"),
        col("ingestion_date")
    )
)

# Deduplicação por ID (última ingestão vence)
windowSpec = Window.partitionBy("launch_id").orderBy(col("ingestion_date").desc()) 
df_silver_launches = (
    df_silver_launches
    .withColumn("rn", row_number().over(windowSpec))
    .filter(col("rn") == 1)
    .drop("rn")
)

df_silver_launches.write.format("delta").mode("overwrite").saveAsTable("workspace.silver.spacex_launches")
print("✅ Silver salvo para launches")

# ====================================================
# 5. Gold – enriquecimento (join com rockets)
# ====================================================
df_rockets = spark.table("workspace.bronze.spacex_rockets").select( # Lê dados de rockets (bronze) e renomea as colunas
    col("id").alias("rocket_id"),
    col("name").alias("rocket_name"),
    col("type").alias("rocket_type"),
    col("active"),
    col("stages")
)

df_gold = (
    df_silver_launches.join(df_rockets, df_silver_launches.rocket == df_rockets.rocket_id, "left")
    .select( #Seleciona colunas finais
        "launch_id",
        "mission_name",
        "date_utc",
        "rocket_name",
        "rocket_type",
        "active",
        "stages",
        "ingestion_date"
    )
)

df_gold.write.format("delta").mode("overwrite").saveAsTable("workspace.gold.spacex_launches")
print("✅ Gold salvo para launches")

# ====================================================
# 6. Exemplo de consulta
# ====================================================
display(
    spark.sql("""
    SELECT mission_name, date_utc, rocket_name
    FROM workspace.gold.spacex_launches
    ORDER BY date_utc DESC
    LIMIT 10
    """)
)
